In [1]:
from test_model import TestModel
import torch

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = TestModel().to(device)
model.eval()
x = torch.randn(16, 2).to(device)

In [2]:
out_fp32 = model(x)

In [3]:
def print_param_dtype(model):
    for name, param in model.named_parameters():
        print(f"{name} is loaded in {param.dtype}")

In [4]:
print(f'{out_fp32.shape}\n')

print_param_dtype(model)
print(out_fp32)

torch.Size([16, 8])

linear1.weight is loaded in torch.float32
linear1.bias is loaded in torch.float32
bn1.weight is loaded in torch.float32
bn1.bias is loaded in torch.float32
linear2.weight is loaded in torch.float32
linear2.bias is loaded in torch.float32
bn2.weight is loaded in torch.float32
bn2.bias is loaded in torch.float32
tensor([[0.8541, 0.1149, 0.1141, 0.0000, 0.1887, 0.2379, 0.0000, 0.4394],
        [0.4347, 0.0000, 0.1950, 0.0000, 0.4956, 0.0000, 0.0000, 0.0000],
        [0.4341, 0.0000, 0.1766, 0.0000, 0.5147, 0.0000, 0.0000, 0.0000],
        [1.0019, 0.3521, 0.0000, 0.0000, 0.4951, 0.2923, 0.0000, 0.1099],
        [1.0963, 0.3803, 0.0000, 0.0000, 0.0000, 0.5342, 0.0000, 0.5385],
        [0.7830, 0.0401, 0.2393, 0.0000, 0.0000, 0.2328, 0.0000, 0.5426],
        [0.6002, 0.0000, 0.2864, 0.0000, 0.2534, 0.0000, 0.0000, 0.4049],
        [0.5767, 0.0000, 0.2725, 0.0000, 0.3464, 0.0000, 0.0000, 0.3579],
        [0.6742, 0.0000, 0.0204, 0.0000, 0.5273, 0.0000, 0.0000, 0.0123],
 

### Cast test model into float16 (Half).
- Casting the entire model to fp16 interacts badly with batch norm layers. 
- float16 is also not supposed on some CPU kernels

In [5]:
model_fp16 = TestModel().to(dtype = torch.float16, device = device) # or TestModel().half()
model_fp16.eval()
x = x.to(torch.float16)

In [6]:
print_param_dtype(model_fp16)
out_fp16 = model_fp16(x)
print(out_fp16)

linear1.weight is loaded in torch.float16
linear1.bias is loaded in torch.float16
bn1.weight is loaded in torch.float16
bn1.bias is loaded in torch.float16
linear2.weight is loaded in torch.float16
linear2.bias is loaded in torch.float16
bn2.weight is loaded in torch.float16
bn2.bias is loaded in torch.float16
tensor([[0.4246, 0.0000, 0.4304, 0.1505, 0.0000, 0.3469, 0.0000, 0.0000],
        [0.4150, 0.0000, 0.2427, 0.0000, 0.0000, 0.1168, 0.0000, 0.0000],
        [0.4092, 0.0000, 0.2111, 0.0000, 0.0000, 0.0774, 0.0000, 0.0000],
        [0.7656, 0.0000, 0.0596, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4512, 0.0000, 0.4309, 0.1252, 0.0000, 0.3513, 0.0000, 0.0000],
        [0.3823, 0.0000, 0.3418, 0.2600, 0.0000, 0.3010, 0.0000, 0.0000],
        [0.4070, 0.0000, 0.4009, 0.1902, 0.0000, 0.3311, 0.0000, 0.0000],
        [0.4192, 0.0000, 0.4302, 0.1558, 0.0000, 0.3459, 0.0000, 0.0000],
        [0.5723, 0.0000, 0.1881, 0.0000, 0.0000, 0.0723, 0.0000, 0.0000],
        [0.3945, 0.000

#### Instead, cast using mixed precision:

In [7]:
with torch.amp.autocast(device):
    out_fp16_autocast = model(x)

print(out_fp16_autocast)

tensor([[0.8540, 0.1149, 0.1140, 0.0000, 0.1888, 0.2378, 0.0000, 0.4390],
        [0.4348, 0.0000, 0.1949, 0.0000, 0.4956, 0.0000, 0.0000, 0.0000],
        [0.4341, 0.0000, 0.1765, 0.0000, 0.5146, 0.0000, 0.0000, 0.0000],
        [1.0020, 0.3521, 0.0000, 0.0000, 0.4951, 0.2925, 0.0000, 0.1100],
        [1.0967, 0.3809, 0.0000, 0.0000, 0.0000, 0.5347, 0.0000, 0.5386],
        [0.7832, 0.0404, 0.2394, 0.0000, 0.0000, 0.2334, 0.0000, 0.5425],
        [0.6006, 0.0000, 0.2861, 0.0000, 0.2537, 0.0000, 0.0000, 0.4048],
        [0.5767, 0.0000, 0.2725, 0.0000, 0.3467, 0.0000, 0.0000, 0.3579],
        [0.6743, 0.0000, 0.0203, 0.0000, 0.5273, 0.0000, 0.0000, 0.0123],
        [0.7778, 0.0338, 0.2205, 0.0000, 0.0494, 0.2075, 0.0000, 0.5088],
        [0.7363, 0.0000, 0.0278, 0.0000, 0.5000, 0.0000, 0.0000, 0.1305],
        [0.6284, 0.0000, 0.0658, 0.0000, 0.5176, 0.0000, 0.0000, 0.0363],
        [0.5996, 0.0000, 0.0719, 0.0000, 0.4763, 0.0000, 0.0000, 0.0000],
        [0.4529, 0.0000, 0.3083, 0.000

- Very close to fp32, but might not be ideal for deeper models

### Cast test model into bfloat16.
- bfloat16 is a more stable, and better alternative to fp16

In [8]:
from copy import deepcopy
model_bf16 = deepcopy(model)
model_bf16 = model_bf16.to(torch.bfloat16)
print_param_dtype(model_bf16)
out_bf16 = model_bf16(x.to(torch.bfloat16))
print(out_bf16)

linear1.weight is loaded in torch.bfloat16
linear1.bias is loaded in torch.bfloat16
bn1.weight is loaded in torch.bfloat16
bn1.bias is loaded in torch.bfloat16
linear2.weight is loaded in torch.bfloat16
linear2.bias is loaded in torch.bfloat16
bn2.weight is loaded in torch.bfloat16
bn2.bias is loaded in torch.bfloat16
tensor([[0.8516, 0.1133, 0.1152, 0.0000, 0.1914, 0.2344, 0.0000, 0.4395],
        [0.4336, 0.0000, 0.1953, 0.0000, 0.4941, 0.0000, 0.0000, 0.0000],
        [0.4336, 0.0000, 0.1768, 0.0000, 0.5156, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.3516, 0.0000, 0.0000, 0.4941, 0.2930, 0.0000, 0.1094],
        [1.0938, 0.3828, 0.0000, 0.0000, 0.0000, 0.5352, 0.0000, 0.5391],
        [0.7812, 0.0410, 0.2412, 0.0000, 0.0000, 0.2344, 0.0000, 0.5469],
        [0.6016, 0.0000, 0.2871, 0.0000, 0.2539, 0.0000, 0.0000, 0.4043],
        [0.5781, 0.0000, 0.2734, 0.0000, 0.3477, 0.0000, 0.0000, 0.3574],
        [0.6719, 0.0000, 0.0215, 0.0000, 0.5234, 0.0000, 0.0000, 0.0117],
        [0.781

### Compute output differences between fp32 and bf16

In [9]:
mean_diff = torch.abs(out_bf16 - out_fp32).mean().item()
max_diff = torch.abs(out_bf16 - out_fp32).max().item()

print(f"Mean diff: {mean_diff} | Max diff: {max_diff}")

Mean diff: 0.0008002215763553977 | Max diff: 0.006844639778137207


### Output logits hold similar values, small differences between the full-precision model and the bf16 model.
- Does not lead to a huge performance degradation, even on large models.